In [1]:
import sys
sys.path.insert(0,"libs")

In [11]:
import pdfplumber
import re

def extract_paragraphs_from_pdf(pdf_path):
    paragraphs = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            paragraphs.append(text)
        
        print("paragraph =", paragraphs)

    return paragraphs

pdf_path = 'mcq.pdf'
paragraphs = extract_paragraphs_from_pdf(pdf_path)
answer = paragraphs[-1]
patterNumber =  r'(\d+\.\s)'
patternA  = r'(?=\b[a-d]\.\s)'

paragraph = []
start_indices =[]
end_indices = []

complete_paragraphs = "".join(paragraphs)
complete_paragraphs = complete_paragraphs.split("Answers")
complete_paragraphs.pop()
complete_paragraphs = "".join(complete_paragraphs)


for pr in re.finditer(patterNumber, complete_paragraphs):
    start_index = pr.start()
    end_index = pr.end()
    start_indices.append(start_index)
    end_indices.append(end_index)

for i in range(1,len(start_indices)-1):
    matched_strng = complete_paragraphs[start_indices[i-1]:start_indices[i]]
    paragraph.append(matched_strng)

paragraph = [re.split(patternA,p) for p in paragraph]
print("p =", paragraph)


valid = [p for p in paragraph if len(p) == 5 ]
invalid = [p for p in paragraph if len(p) != 5]

answer_parts = answer.split("Answers")
all_answer = answer_parts[-1].split("\n***")[0]
all_answer = all_answer.replace("\n"," ")
splits :list[str] = all_answer.split(" ")
splits = [ s for s in splits if len(s.strip()) != 0 ]

all_question = []

for v in valid:
    questions : str = v[0].split(".")
    for s in splits:
        s:str
        number = s.split(".")
        if(questions[0].strip() == number[0].strip()):
            correct = 0,
            if number[1] == "a" : correct = 0
            elif number[1] == "b" : correct = 1
            elif number[1] == "c" : correct = 2
            elif number[1] == "d" : correct = 3
            else : correct = 0
            print(
                questions[1].strip()
            )
            all_question.append([
                questions[1].strip(),
                v[1].replace("a.","").strip(),
                v[2].replace("b.","").strip(),
                v[3].replace("c.","").strip(),
                v[4].replace("d.","").strip(),
                correct,
                1,
                ""
            ])



paragraph = ['Multiple Choice Quesitons\n1. For a good building stone how much is the required crushing strength?\na. Less than 50 N/mm2 b. Greater than 100 N/mm2\nc. 155 N/mm2 d. 10 N/mm2\n2. Which of the following is a good fire-resistant stone?\na. Clay b. Granite c. Quartz d. Limestone\n3. What is a freestone?\na. Stone free from impurities b. Stone that doesn’t require dressing\nc. Metamorphic stone d. Stone free from veins and planes of cleavage\n4. Why are stones with lighter shades of colour preferred?\na. Easy to clean b. Easily available\nc. Don’t spoil the appearance d. Darker shades are heavier\n5. Hardness is an important parameter considered in the construction of:\na. Slabs b. Walls c. Bridges d. Arches\n6. What is the required specific gravity for a good building stone?\na. Greater than 2.7 b. Less than 3 c. Greater than 3 d. Less than 2.7\n7. The percentage absorption by weight of a good stone, after how many hours should not exceed .6?\na. 6 hrs b. 12 hrs c. 48 hrs d.

In [10]:
import csv

file_path = "mcq_questions.csv"

with open(file_path,mode="w",newline= "\n") as file:
    writer = csv.writer(file)
    writer.writerow(["Question", "answer", "answer", "answer", "answer", "correct", "marks", "description"])
    writer.writerows(all_question)